# Business Entity Recognition Demo

This notebook is designed to demonstrate how easy it is to use the SAP AI Business Services - Business Entity Recognition service for classification tasks. In this demo, we will do inference for a plain text using a pretrained model.

For the demo, we prepared this Jupyter Notebook which demonstrates the use of this client library to invoke the most important functions of the Business Entity Recognition REST API. 

## Settings

The settings under `Environment specific configuration` require a valid service key for the Business Entity Recognition service on SAP Cloud Plattform.

The keys in the service key needed here are named exactly as the variables, specifically:
- url: The URL of the service deployment provided in the outermost hierachy of the service key json file
- uaa_url: The URL of the UAA server used for authentication provided in the __uaa__ of the service key json file
- uaa_clientid: The clientid used for authentication to the UAA server provided in the __uaa__ of the service key json file
- uaa_clientsecret: The clientsecret used for authentication to the UAA server provided in the __uaa__ of the service key json file

Service key json has to be copied to config.json file.

In [ ]:
# update working directory path

import os

os.chdir('../')

#print(os.getcwd())

import pathlib
pathlib.Path().absolute()

In [ ]:
from sap_ber_client import ber_api_client
from pprint import pprint

In [ ]:
import importlib
importlib.reload(ber_api_client)

In [ ]:
import json
config_file = os.getcwd() + '/examples/config.json'
with open(config_file, 'rb') as config:
    config_json = json.load(config)
    config.close()
    
#config_json

In [ ]:
# Instaniate object used to communicate with DC REST API

url = config_json['url']
uaa_clientid = config_json['uaa']['clientid']
uaa_clientsecret = config_json['uaa']['clientsecret']
uaa_url = config_json['uaa']['url']

my_ber_client = ber_api_client.BER_API_Client(url, uaa_clientid, uaa_clientsecret, uaa_url)

#print(my_ber_client.base_url)

## Display access token

In [ ]:
# Token can be used to interact with e.g. swagger UI to explore BER API
#print(my_ber_client.session.headers)
#print("\nYou can use this token to Authorize here and explore the API via Swagger UI: \n{}api/v1/".format(url))

## Inference


In [ ]:
# post inference job with sap_email_business_entity

#predefined model details as below
model_name = "sap_email_business_entity"
model_version = 1


text = 'Hello, I would like to know the status of the invoice 456789. Regards, John'
response = my_ber_client.post_inference_job(text, model_name, model_version)
pprint(response.json())
inference_jobid = response.json()["data"]["id"]

In [ ]:
#Get inference result 
response = my_ber_client.get_inference_job(inference_jobid)
pprint(response.json())

In [ ]:
# post inference job with sap_invoice_header

#predefined model details as below
model_name = "sap_invoice_header"
model_version = 1


text = 'Order number 12345678 Order type Sales Force Order abcdef Date 01/01/2001 Customer number 9876543 Customer service 123/ 456 6789 E-mail eshop@gmail.com  Your Reference: Order receipt confirmation Customer address 508 W. St Margarets St. Brooklyn, NY 11228 Billing address 508 W. St Margarets St. Brooklyn, NY 11228 Ordered by Fenton Moon Shipping type: UPS 2nd Day Air PM Terms of payment: 30 Days Net due  Your message: Item Material   Description Qty.        Price per Unit Net  steel        gray / Length 12 mm / 1 35.00 35.00  Gross Amount USD 35.00 Freight/ packaging USD 15.00 Tax 2 % USD 1 Tax 4 % USD 2 Total amount USD 53'
response = my_ber_client.post_inference_job(text, model_name, model_version)
pprint(response.json())
inference_jobid = response.json()["data"]["id"]

In [ ]:
#Get inference result 
response = my_ber_client.get_inference_job(inference_jobid)
pprint(response.json())

## Batch Inference

In [ ]:
# Create Inference dataset
response = my_ber_client.create_dataset("inference")
pprint(response.json())
inference_dataset_id = response.json()["data"]["datasetId"]
print(inference_dataset_id)
inference_dataset_folder = os.getcwd() + '/examples/data/pretrained_batch_inference.json'
print("Uploading inference documents to the dataset")
response = my_ber_client.upload_document_to_dataset(inference_dataset_id, inference_dataset_folder)
print("Finished uploading inference documents to the dataset")
pprint(response.json())

# post batch inference job
response = my_ber_client.post_batch_inference_job(inference_dataset_id, model_name, model_version)
pprint(response.json())
batch_inference_job_id = response.json()["data"]["id"]


In [ ]:
#Get inference result
response = my_ber_client.get_inference_job(batch_inference_job_id)
pprint(response.json())

#get inference batch job result
response = my_ber_client.get_batch_inference_job_result(batch_inference_job_id)

pprint(response.json())

## Delete Datasets

In [ ]:
response = my_ber_client.delete_dataset(inference_dataset_id)
pprint(response.json())